# Figures associated to fibroblast processing

**You need to tun the notebook 2_ first!!!!**

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk
import itertools

In [ ]:
from functions import adata_plot_KOvsWT
from functions import dict_WT_KO_colors

In [ ]:
from datetime import date

today = date.today()
today = today.strftime("%Y_%m_%d")

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 200

pd.set_option('display.max_columns', None)

## Functions and variables

In [ ]:
%store -r dict_WT_KO_colors
%store -r seed

## Load adata

In [ ]:
adata_all_harmony = sc.read_h5ad('adatas/adata_all_harmony.h5')
adata_fb = adata_all_harmony[adata_all_harmony.obs['merged_cell_type'] == 'Fibroblast']

In [ ]:
sc.pp.filter_genes(adata_fb, min_counts=1)

In [ ]:
sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_fb)

sc.pp.pca(adata_fb, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_fb, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_fb, random_state=seed, n_neighbors=int(len(adata_fb) ** 0.5 * 0.5), metric='cosine', use_rep='X_pca_harmony')

We see that some populations (*Coch*$^+$ or *Thbs4*$^+$) are separated into two populations. We are going to separate them by finding the corresponding DEGs.

In [ ]:
dict_fbs = {    'Fibroblast Ptgs2$^+$': ['Ptgs2', 'Il1rl1', 'Nr4a2', 'Pdpn', 'Gxylt2', 'Lum', 'Mmp2', 'Olfr1033', 'Arpc3', 'Ptges', 'Plin2', 'Npc2', 'Tnc', 'B2m', 'Twist2'],
                'Fibroblast Hilpda$^+$': ['Hilpda', 'Mt2', 'Ero1l', 'Prg4', 'Ldha', 'Mpp6', 'Rnf126', 'Pfkp', 'Slc7a2', 'Rcan1'], 
                'Fibroblast Cxcl1$^+$': ['Cxcl1', 'Ccl7', 'Has1', 'Cxcl2', 'Tnfaip6', 'Cxcl10', 'Ccl2', 'Gfpt2', 'Il6', 'Gpc3', 'Ccl11', 'Clec2d'],
                'Fibroblast Cxcl12$^+$': ['Cxcl12', 'Lpl', 'Cygb', 'Ccl19', 'Htra3', 'C1s1',  'Ly6a', 'Plac9b', 'Gpc3', 'Itm2a', 'Cpxm1', 'Efemp1', 'H19', 'Zim1'],
                'Fibroblast Ccn5$^+$': ['Ccn5', 'Phldb2', 'Thbs2', 'Igfbp2', 'Col8a1', 'Eid1', 'Arap1', 'Gpr153', 'Col12a1', 'Lrrc15', 'Osr2', 'Pth1r', 'C1qtnf3'],
                'Fibroblast Ltbp2$^+$': ['Ltbp2', 'Thbs4', 'Ecrg4', 'Eln', 'Ccdc3', 'Chodl', 'Edn3', 'Wnt5a', 'Cilp', 'Crabp1', 'Igfbp2',  'Ism1', 'a', 'Slit2', 'Agt', 'Sema3a', 'Rerg'],
                'Fibroblast Clec3b$^+$': ['Clec3b', 'Pi16', 'Scara5', 'Entpd2', 'Fbn1',  'Ugp2', 'Cd248', 'Sema3c', 'Adamtsl4', 'Mgst3'],
                'Fibroblast Cilp2$^+$': ['Cilp2', 'Chad', 'Egfl6', 'Comp', 'Fmod', 'Tnmd', 'Ecrg4', 'Kera', 'Acan', 'Scx', 'Fibin', 'Tspan2'], 
                'Fibroblast Coch$^+$': ['Coch', 'Emid1', 'Gldn', 'Ntn5', 'Tnmd', 'Col16a1', 'Col11a1',  'Crabp1', 'Mafb', 'Fmod', 'Fbn2', 'Cxcr4', 'Ogn', 'Kera', 'Wfdc1', 'Hs3st6', 'Pi16', 'Trib2'],
                'Fibroblast Rab37$^+$': ['Rab37', 'Col22a1', 'F13a1', 'Htra4', 'Tspan15', 'Rgcc', 'Hbegf', 'Cd55', 'Capg', 'Itga6', 'Pcsk6', 'Sema3c', 'Htra4', 'Sox5', 'Rgs5', 'Aldh1a3', 'Sema3d', 
                                         'Cmtm8', 'Ackr2'],               
                'Fibroblast Cfh$^+$': ['Cfh', 'Alpl', 'Lifr', 'Sp7', 'Spp1', 'Runx2', 'Cp', 'Mmp13', 'Ptprd', 'Hp',  'Ibsp', 'Cadm1', 'Lgmn', 'Limch1', 'Vdr', 'Dapk2', 'Dlx5', 'Sept4'],
                'Fibroblast Serpine2$^+$': ['Serpine2', 'Shox2', 'Wif1', 'Col23a1', 'Gm48159', 'Cd24a', 'Nav2', 'Tfap2c', 'Tanc1', 'Scube3', 'Cblb', 'Fgfr2', 'Olfm', 'Prdm1', 'Alx3', 'Inhba', 'Dio3', 'Rspo1'],
                  }

In [ ]:
dict_fbs_colors = {  'Fibroblast Ccn5$^+$': '#d3bc5f',
                     'Fibroblast Cfh$^+$': '#229eb2',
                     'Fibroblast Cilp2$^+$': '#8e0c6f',
                     'Fibroblast Clec3b$^+$': '#ff5599',
                     'Fibroblast Coch$^+$': '#4d0765',
                     'Fibroblast Cxcl12$^+$': '#ff9955',
                     'Fibroblast Cxcl1$^+$': '#a05a2c',
                     'Fibroblast Hilpda$^+$': '#702a17',
                     'Fibroblast Ltbp2$^+$': '#ae54b8',
                     'Fibroblast Ptgs2$^+$': '#67dcf1',
                     'Fibroblast Rab37$^+$': '#d40055',
                     'Fibroblast Serpine2$^+$': '#20668d'}

In [ ]:
sc.tl.leiden(adata_fb, resolution=4)
assign_cats(adata_fb, dict_cats=dict_fbs, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')

adata_fb.uns['cell_type_colors'] = list(dict_fbs_colors.values())

## UMAP of fibroblast subpopulations and batch

In [ ]:
sc.tl.umap(adata_fb)
sc.pl.umap(adata_fb, color=['batch', 'cell_type'], ncols=2, frameon=False, cmap=magma)

## Proportion of cell types within WT and KO

In [ ]:
adata_plot_KOvsWT(adata_fb, list(dict_fbs.keys()), col_cell_type='cell_type')

## Table with markers and p-values

In [ ]:
sc.tl.rank_genes_groups(adata_fb, groupby='cell_type')
names = pd.DataFrame(adata_fb.uns['rank_genes_groups']['names'])
pvals = pd.DataFrame(adata_fb.uns['rank_genes_groups']['pvals_adj'])

In [ ]:
# Create a DataFrame with 30 most important genes
list_dfs = []
for cell_type, genes in dict_fbs.items():
    df = pd.DataFrame({f'{cell_type} genes': names[cell_type], f'p-value {cell_type}': pvals[cell_type]}, dtype=object)
    df_short = df[df[f'{cell_type} genes'].isin(genes)].sort_values(by=f'p-value {cell_type}', ascending=True).iloc[:30].reset_index(drop=True)
    df[f'p-value {cell_type}'] = df[f'p-value {cell_type}'].apply(lambda x: '%.3E' % x).values.tolist()
    list_dfs.append(df_short)

In [ ]:
df_all_celltypes = pd.concat(list_dfs, axis=1)
df_all_celltypes.to_csv(f'results/{today}-markers_4_pops.csv', index=False)
df_all_celltypes

## Dotplot with markers of fibroblast subtypes

In [ ]:
sc.pl.dotplot(adata_fb, groupby='cell_type', var_names=df_all_celltypes.iloc[:4,::2].values.transpose().ravel(), dendrogram=False, show=False, 
             categories_order=list(dict_fbs.keys()))

## Heatmaps of relevant pathways

In [ ]:
complement_genes = ['C1qa', 'C1qb', 'C1qbp', 'C1qc', 'C1ra', 'C1rb', 'C1rl', 'C1s1', 'C1s2', 'C2', 'C3', 'C4a', 'C4b', 'C8g', 'Cd46', 'Cd55', 'Cd55b', 'Cd59a', 'Cd59b', 'Cfb', 'Cfd', 'Cfh', 
                    'Cfhr1', 'Cfhr2', 'Cfp', 'Clu', 'Colec10', 'Colec11', 'Cr1l', 'Cr2', 'Crp', 'Csmd3', 'F3', 'Fcna',  'Il1b', 'Krt1', 'Masp1', 'Masp2', 'Phb', 'Procr', 'Rgcc', 
                    'Serping1', 'Susd4', 'Thbd', 'Trbc1', 'Trbc2', 'Trdc', 'Trdv4', 'Trem2']  # GO:0006956

complement_genes_filtered = ['C1qbp', 'C1s1', 'C3', 'C4b', 'Cd55', 'Cd55b', 'Cfh', 'Clu', 'F3', 'Masp1', 'Phb', 'Rgcc', 'Serping1', 'Thbd']



cytokine_interaction_genes = ['Ackr2', 'Ackr3', 'Ackr4', 'Adam17', 'Amh', 'Angpt1', 'Angpt2', 'Angpt4', 'Babam2', 'Bdnf', 'Bex3', 'Bmp3', 'Bmp4', 'Bmp8a', 'Cadm4', 'Casp3', 'Casp8', 'Ccdc88a', 'Ccl11', 
                              'Ccl12', 'Ccl19', 'Ccl2', 'Ccl20', 'Ccl21a', 'Ccl24', 'Ccl25', 'Ccl27a', 'Ccl28', 'Ccl3', 'Ccl4', 'Ccl5', 'Ccl6', 'Ccl7', 'Ccl8', 'Ccl9', 'Ccr1', 'Ccr10', 'Ccr5', 'Ccr7', 
                              'Ccr9', 'Ccrl2', 'Cd2ap', 'Cd300lf', 'Cd4', 'Cd44', 'Cd70', 'Cd74', 'Cdh5', 'Ceacam1', 'Cflar', 'Cklf', 'Clcf1', 'Cnih4', 'Cntfr', 'Creb3', 'Crlf1', 'Crlf2', 'Csf1', 'Csf2', 
                              'Csf2ra', 'Csf2rb', 'Csf2rb2', 'Csf3', 'Csf3r', 'Ctf1', 'Ctf2', 'Cx3cl1', 'Cx3cr1', 'Cxcl1', 'Cxcl10', 'Cxcl11', 'Cxcl12', 'Cxcl13', 'Cxcl14', 'Cxcl15', 'Cxcl16', 'Cxcl2', 
                              'Cxcl3', 'Cxcl5', 'Cxcl9', 'Cxcr2', 'Cxcr4', 'Cxcr5', 'Cxcr6', 'Dab2ip', 'Ddt', 'Defb1', 'Defb6', 'Dnaja3', 'Ecm1', 'Eda', 'Edaradd', 'Efna5', 'Eng', 'Epor', 'Erap1', 'F3', 
                              'Fadd', 'Fem1b', 'Fermt2', 'Fkbp1a', 'Frs2', 'Fzd4', 'Gata3', 'Gdnf', 'Gfra2', 'Gfra4', 'Ghr', 'Gm13272', 'Gm13283', 'Gm13305', 'Gm21970', 'Gpr35', 'Gpr75', 'Grb2', 'Grem1', 
                              'Grin2b', 'Hsd17b7', 'Ifnar1', 'Ifnar2', 'Ifngr1', 'Ifngr2', 'Ifnlr1', 'Ifnz', 'Il10', 'Il10ra', 'Il10rb', 'Il11', 'Il11ra1', 'Il12a', 'Il12rb1', 'Il12rb2', 'Il13ra1', 'Il13ra2', 
                              'Il15', 'Il15ra', 'Il16', 'Il17f', 'Il17ra', 'Il17rb', 'Il17rc', 'Il17rd', 'Il17re', 'Il18', 'Il18r1', 'Il18rap', 'Il1b', 'Il1r1', 'Il1r2', 'Il1rap', 'Il1rl1', 'Il1rl2', 
                              'Il1rn', 'Il20ra', 'Il20rb', 'Il21r', 'Il23a', 'Il27ra', 'Il2ra', 'Il2rg', 'Il31ra', 'Il33', 'Il34', 'Il3ra', 'Il4ra', 'Il6', 'Il6ra', 'Il6st', 'Il7', 'Irak1', 'Irak4', 
                              'Itch', 'Itgb3', 'Jak1', 'Jak2', 'Jak3', 'Kitl', 'Lefty1', 'Lefty2', 'Lepr', 'Lif', 'Lifr', 'Lta', 'Ltb', 'Lyn', 'Mif', 'Mpl', 'Msmp', 'Myd88', 'Nars', 'Nes', 'Ngf', 'Ngfr', 
                              'Nol3', 'Nradd', 'Nsmaf', 'Ntf3', 'Ntf5', 'Ntrk1', 'Nucb2', 'Nup85', 'Osm', 'Osmr', 'Pdcl3', 'Pf4', 'Pgf', 'Pibf1', 'Pik3r1', 'Plcg1', 'Prdm4', 'Prl2c3', 'Prl2c5', 'Prl7d1', 
                              'Prlr', 'Ptpn6', 'Pycard', 'Rasl11b', 'Ripk1', 'Rnf41', 'S100a14', 'Sdcbp', 'Sh2b3', 'Shc1', 'Siva1', 'Smad2', 'Smad3', 'Smad6', 'Smad7', 'Smurf2', 'Snx25', 'Snx6', 'Socs2', 
                              'Spred1', 'Spred2', 'Spred3', 'Stap1', 'Stat1', 'Stat3', 'Syk', 'Tgfb1', 'Tgfb2', 'Tgfb3', 'Tgfbr1', 'Tgfbr2', 'Tgfbr3', 'Tgfbr3l', 'Tgfbrap1', 'Timm50', 'Tlr5', 'Tmbim1', 
                              'Tnf', 'Tnfsf10', 'Tnfsf11', 'Tnfsf12', 'Tnfsf13b', 'Tnfsf15', 'Tnfsf4', 'Tnfsf8', 'Tnfsf9', 'Tollip', 'Tradd', 'Traf1', 'Traf2', 'Traf3', 'Traf4', 'Traf5', 'Traf6', 'Trim37', 
                              'Trip6', 'Tslp', 'Tyk2', 'Usp15', 'Vegfa', 'Vegfb', 'Vegfc', 'Vegfd', 'Xcl1', 'Xcr1', 'Zfp110', 'Zfp369']  # GO:0005126 + GO:0004896

cytokine_interaction_genes_filtered = ['Ackr2', 'Adam17', 'Bex3', 'Bmp4', 'Casp3', 'Ccl11', 'Ccl19', 'Ccl2', 'Ccl5', 'Ccl7', 'Cd2ap', 'Cd44', 'Cflar', 'Cnih4', 'Crlf1', 'Csf1', 'Cxcl1', 'Cxcl10', 'Cxcl12',
                                       'Cxcl14', 'Cxcl2', 'Cxcl5', 'Dab2ip', 'Ddt', 'Ecm1', 'Eng', 'Fermt2', 'Fkbp1a', 'Ghr', 'Ifnar2', 'Ifngr1', 'Il11ra1', 'Il13ra1', 'Il1r1', 'Il1rl1', 'Il33', 
                                       'Il6', 'Il6st', 'Irak1', 'Itch', 'Jak1', 'Jak2', 'Kitl', 'Lepr', 'Lif', 'Lifr', 'Mif', 'Nars', 'Nucb2', 'Pdcl3', 'Pgf', 'Pik3r1', 'Rasl11b', 'Sdcbp', 'Siva1', 
                                       'Smad7', 'Snx6', 'Stat3', 'Tgfb1', 'Tgfb2', 'Tgfb3', 'Tgfbr3', 'Timm50', 'Tmbim1', 'Tnfsf12', 'Tnfsf9', 'Tollip', 'Trip6', 'Vegfa',]



cytokine_release = ['Csf1', 'Csf2', 'Ccl2', 'Cxcl1', 'Il11']




ECM_assembly_disassembly = ['Adamts15', 'Adamts5', 'Agt', 'Antxr1', 'Atp7a', 'Carmil2', 'Clasp1', 'Clasp2', 'Cma1', 'Col1a2', 'Col3a1', 'Ctsg', 'Ctss', 'Dag1', 'Ddr1', 'Ddr2', 'Dpp4', 'Efemp2', 
                            'Emilin1', 'Eng', 'Ets1', 'Exoc8', 'Fap', 'Fbln5', 'Fgfr4', 'Fkbp10', 'Flot1', 'Fscn1', 'Gpm6b', 'Hapln2', 'Has1', 'Has2', 'Has3', 'Hpn', 'Hpse', 'Ihh', 'Il6', 
                            'Kif9', 'Klk5', 'Lamb1', 'Lamb2', 'Lamb3', 'Lamc1', 'Lcp1', 'Lox', 'Ltbp3', 'Ltbp4', 'Meltf', 'Mfap4', 'Mmp13', 'Myh11', 'Notch1', 'Noxo1', 'Ntn4', 'Ntng1', 'Ntng2', 
                            'Pbxip1', 'Pdpn', 'Phldb1', 'Phldb2', 'Plod3', 'Pxdn', 'Qsox1', 'Ramp2', 'Rgcc', 'Sema5a', 'Sh3pxd2b', 'Smad3', 'Smpd3', 'Sox9', 'Tgfb1', 'Tgfb2', 'Thsd4', 
                            'Tie1', 'Tnxb'] # GO:0022617 + GO:0085029

ECM_assembly_disassembly_filtered = ['Agt', 'Antxr1', 'Col1a2', 'Col3a1', 'Dag1', 'Ddr2', 'Efemp2', 'Emilin1', 'Eng', 'Ets1', 'Fap', 'Fbln5', 'Fkbp10', 'Flot1', 'Fscn1', 'Gpm6b', 'Has1', 
                                     'Il6', 'Lamb1', 'Lamb2', 'Lamc1', 'Lox', 'Ltbp3', 'Ltbp4', 'Mfap4', 'Mmp13', 'Ntn4', 'Pbxip1', 'Pdpn', 'Phldb2', 'Pxdn', 'Qsox1', 'Rgcc', 'Sema5a', 'Sh3pxd2b', 
                                    'Smpd3', 'Tgfb1', 'Tgfb2', 'Tnxb' ]




hypoxia = ['Acaa2', 'Adam8', 'Ado', 'Adrb2', 'Ajuba', 'Ak4', 'Akt1', 'Aqp1', 'Aqp3', 'Atf2', 'Bad', 'Bcl2', 'Bmyc', 'Bnip3', 'Bnip3l', 'Cbs', 'Cd34', 'Cited2', 'Commd1', 'Cpeb1', 'Cpeb2', 'Cr1l', 
           'Cybb', 'Ddah1', 'Egln1', 'Egln2', 'Egln3', 'Egr1', 'Eif4ebp1', 'Eno1', 'Eno1b', 'Ep300', 'Epas1', 'Epha4', 'Fam162a', 'Fmn2', 'Fndc1', 'Gata6', 'Gnb1', 'Hif1a', 'Hif1an', 'Hif3a', 
           'Higd1a', 'Hp1bp3', 'Hyou1', 'Kcnk2', 'Kcnk3', 'Kdm6a', 'Map2k1', 'Mgarp', 'Mir214', 'Mlst8', 'Mtor', 'Myc', 'Ndnf', 'Nfe2l2', 'Nol3', 'Nop53', 'Notch1', 'Npepps', 'Ogt', 'P4hb', 
           'Pdk1', 'Pdk3', 'Pgk1', 'Pik3cb', 'Pink1', 'Plk3', 'Ppard', 'Pparg', 'Prkce', 'Pten', 'Ptgis', 'Rbpj', 'Rgcc', 'Rock2', 'Rora', 'Rptor', 'Rtn4', 'Rwdd3', 'Scn2a', 'Sdhd', 'Sirt1', 
           'Sirt2', 'Slc2a4', 'Slc8a3', 'Slc9a1', 'Stox1', 'Stub1', 'Suv39h1', 'Suv39h2', 'Tbl2', 'Tert', 'Tigar', 'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 'Vegfa', 'Vhl', 'Vldlr', 
           'Zfas1', 'Zfp36l1']  # GO:0071456

hypoxia_filtered = ['Acaa2', 'Akt1', 'Aqp1', 'Bad', 'Bcl2', 'Bmyc', 'Bnip3', 'Bnip3l', 'Cd34', 'Commd1', 'Cr1l', 'Egln1', 'Egr1', 'Eif4ebp1', 'Eno1', 'Ep300', 'Fam162a', 'Fndc1', 'Gnb1', 'Hif1a', 
                    'Higd1a', 'Hp1bp3', 'Myc', 'Nfe2l2', 'Nop53', 'Npepps', 'Ogt', 'P4hb', 'Pgk1', 'Pink1', 'Ppard', 'Pten', 'Ptgis', 'Rbpj', 'Rgcc', 'Rora', 'Rtn4', 
                    'Sdhd', 'Sirt1', 'Sirt2', 'Slc9a1', 'Tmbim6', 'Trp53', 'Twist1', 'Ubqln1', 'Usp19', 'Vasn', 'Vegfa', 'Vhl', 'Vldlr', 'Zfas1', 'Zfp36l1']




OXPHOS = ['1700066M21Rik', 'Abcd1', 'Actn3', 'Afg1l', 'Ak4', 'Atp5a1', 'Atp5b', 'Atp5c1', 'Atp5d', 'Atp5e', 'Atp5h', 'Atp5j', 'Atp5j2', 'Atp5k', 'Atp5o', 'Atp7a', 'Atpsckmt', 'Bdnf', 'Bid', 'Ccnb1', 
          'Cdk1', 'Chchd10', 'Coa6', 'Coq7', 'Coq9', 'Cox4i1', 'Cox4i2', 'Cox5a', 'Cox5b', 'Cox6a1', 'Cox6a2', 'Cox6c', 'Cox7a1', 'Cox7a2', 'Cox7a2l', 'Cox7c', 'Cox8a', 'Cox8b', 'Cyc1', 'Cycs', 
          'Dguok', 'Dld', 'Dnajc15', 'Dnajc30', 'Fxn', 'Iscu', 'Mlxipl', 'Msh2', 'mt-Atp6', 'mt-Atp8', 'mt-Co1', 'mt-Co2', 'mt-Co3', 'mt-Cytb', 'mt-Nd1', 'mt-Nd2', 'mt-Nd3', 'mt-Nd4', 'mt-Nd4l', 
          'mt-Nd5', 'mt-Nd6', 'Mtch2', 'Myc', 'Myog', 'Ndufa1', 'Ndufa2', 'Ndufa3', 'Ndufa5', 'Ndufa6', 'Ndufa7', 'Ndufa8', 'Ndufa9', 'Ndufa10', 'Ndufa11', 'Ndufa12', 'Ndufa13', 'Ndufab1', 'Ndufaf1',
          'Ndufb2', 'Ndufb3', 'Ndufb4', 'Ndufb5', 'Ndufb6', 'Ndufb7', 'Ndufb8', 'Ndufb9', 'Ndufb10', 'Ndufb11', 'Ndufc1', 'Ndufc2', 'Ndufs1', 'Ndufs2', 'Ndufs3', 'Ndufs4', 'Ndufs5', 'Ndufs6', 'Ndufs7', 
          'Ndufs8', 'Ndufv1', 'Ndufv2', 'Ndufv3', 'Nipsnap2', 'Nupr1', 'Park7', 'Pde2a', 'Pde12', 'Pink1', 'Ppif', 'Rhoa', 'Sdha', 'Sdhaf2', 'Sdhb', 'Sdhc', 'Sdhd', 'Shmt2', 'Slc25a23', 'Slc25a33', 
          'Snca', 'Stoml2', 'Tefm', 'Tnf', 'Uqcc2', 'Uqcc3', 'Uqcr10', 'Uqcr11', 'Uqcrb', 'Uqcrc1', 'Uqcrc2', 'Uqcrfs1', 'Uqcrh', 'Uqcrq', 'Vcp']  # GO:0006119

OXPHOS_filtered = ['Atp5a1', 'Atp5b', 'Atp5c1', 'Atp5d', 'Atp5e', 'Atp5h', 'Atp5j', 'Atp5j2', 'Atp5k', 'Chchd10', 'Cox4i1', 'Cox4i2', 'Cox5a', 'Cox5b', 'Cox6a1', 'Cox6c', 'Cox7a2', 'Cox7a2l', 
                   'Cox7c', 'Cox8a', 'Cyc1', 'Dnajc15', 'Iscu', 'mt-Atp6', 'mt-Atp8', 'mt-Co1', 'mt-Cytb', 'mt-Nd1', 'mt-Nd2', 'mt-Nd3', 'mt-Nd4', 'mt-Nd4l', 
                   'mt-Nd5', 'mt-Nd6', 'Mtch2', 'Myc','Ndufa1', 'Ndufa2', 'Ndufa3', 'Ndufa5', 'Ndufa6', 'Ndufa7', 'Ndufa10', 'Ndufa11', 'Ndufa13', 'Ndufb2', 'Ndufb3', 'Ndufb5', 'Ndufb7', 
                   'Ndufb9', 'Ndufb10', 'Ndufb11', 'Ndufc2', 'Nupr1', 'Park7', 'Rhoa', 'Sdhb', 'Sdhd', 'Shmt2', 'Stoml2', 'Uqcc2', 'Uqcr10', 'Uqcr11', 'Uqcrb', 'Uqcrq' ]



glycolysis = ['Actn3', 'Adpgk', 'Aldoa', 'Aldoart1', 'Aldob', 'Aldoc', 'App', 'Arl2', 'Bpgm', 'Cbfa2t3', 'Ddit4', 'Dhtkd1', 'Eif6', 'Eno1', 'Eno1b', 'Eno2', 'Eno3', 'Enpp1', 'Ep300', 'Fbp1', 'Flcn', 
              'Foxk1', 'Foxk2', 'Gale', 'Galk1', 'Galt', 'Gapdh', 'Gapdhs', 'Gck', 'Git1', 'Gpd1', 'Gpi1', 'Hdac4', 'Hif1a', 'Hk1', 'Hk2', 'Hk3', 'Hkdc1', 'Htr2a', 'Ier3', 'Igf1', 'Insr', 'Jmjd8', 
              'Khk', 'Mlst8', 'Mlxipl', 'Mpi', 'Mtch2', 'Mtor', 'Myc', 'Myog', 'Ncor1', 'Nupr1', 'Ogdh', 'Ogt', 'P2rx7', 'Pfkfb2', 'Pfkl', 'Pfkm', 'Pfkp', 'Pgam1', 'Pgk1', 'Pkm', 'Ppara', 'Ppargc1a', 
              'Prkaa1', 'Prkaa2', 'Prkag2', 'Prkag3', 'Prxl2c', 'Psen1', 'Rptor', 'Sirt6', 'Slc2a6', 'Slc4a1', 'Slc4a4', 'Stat3', 'Tigar', 'Tkfc', 'Tpi1', 'Zbtb7a', 'Zbtb20']  # GO:0006096


glycolysis_filtered = ['Aldoa', 'App', 'Arl2', 'Ddit4', 'Eif6', 'Eno1', 'Enpp1', 'Ep300', 'Flcn', 'Galk1', 'Gapdh', 'Git1', 'Gpi1', 'Hdac4', 'Hif1a', 'Hk1', 'Ier3', 'Igf1', 'Mtch2', 'Myc', 'Ncor1', 
                       'Nupr1', 'Ogdh', 'Ogt', 'Pfkp', 'Pgam1', 'Pgk1', 'Pkm', 'Prxl2c', 'Stat3', 'Tpi1', 'Zbtb7a', 'Zbtb20' ]



stress = ['Abcd1', 'Abl1', 'Adnp2', 'Agap3', 'Aif1', 'Aifm2', 'Akr1b3', 'Akr1c14', 'Akr1c18', 'Akr1cl', 'Akt1', 'Aldh2', 'Aldh3b1', 'Alox5', 'Ankzf1', 'Anxa1', 'Apex1', 'App', 'Aqp1', 'Arl6ip5', 
          'Arnt', 'Arntl', 'Atf2', 'Atf4', 'Atg7', 'Atp13a2', 'Atp2a2', 'Atp7a', 'Axl', 'Bag5', 'Bcl2', 'Bmp4', 'Bnip3', 'Brf2', 'Btk', 'Capn1', 'Cat', 'Ccs', 'Cd36', 'Cdkn2a', 'Cfl1', 'Chchd4', 
          'Chd6', 'Chuk', 'Coq7', 'Cpeb2', 'Ctnnb1', 'Cyp1b1', 'Dapk1', 'Ddias', 'Ddr2', 'Ddx3x', 'Dhfr', 'Diablo', 'Dnajc3', 'Ect2', 'Edn1', 'Ednra', 'Egfr', 'Eif2ak1', 'Eif2ak3', 'Eif2ak4', 
          'Eif2s1', 'Eif4g1', 'Endog', 'Epas1', 'Epor', 'Ercc6l2', 'Ermp1', 'Etv5', 'Ezh2', 'Fancc', 'Fancd2', 'Fbln5', 'Fbxo7', 'Fbxw7', 'Fech', 'Fer', 'Fos', 'Foxa1', 'Foxo1', 'Foxo3', 'Foxp1', 
          'Fut8', 'Fxn', 'Fyn', 'Fzd1', 'G6pdx', 'Gata4', 'Gch1', 'Gfer', 'Gpr37l1', 'Gpx1', 'Gskip', 'Gsr', 'Hdac2', 'Hdac6', 'Hgf', 'Hif1a', 'Hk3', 'Hmox1', 'Hsf1', 'Hspb1', 'Hsph1', 'Htra2', 
          'Il10', 'Il6', 'Impact', 'Itgam', 'Jak2', 'Jun', 'Kdm6b', 'Keap1', 'Krt13', 'Lancl1', 'Lcn2', 'Lonp1', 'Lrrk2', 'Map3k5', 'Mapk1', 'Mapk13', 'Mapk3', 'Mapk7', 'Mapk8', 'Mapk9', 'Mapkap1', 
          'Mapt', 'Mcl1', 'Meak7', 'Melk', 'Met', 'Mgat3', 'Mgst1', 'Mmp2', 'Mmp3', 'Mmp9', 'Mpo', 'Mpv17', 'Msra', 'Mt3', 'Nck1', 'Nck2', 'Ncoa7', 'Net1', 'Nfe2l1', 'Nfe2l2', 'Ngb', 'Ngfr', 
          'Nme5', 'Nol3', 'Nono', 'Nos3', 'Nox1', 'Npm1', 'Nqo1', 'Nr4a2', 'Nr4a3', 'Oser1', 'Oxr1', 'P4hb', 'Park7', 'Parp1', 'Pawr', 'Pcgf2', 'Pdcd10', 'Pde8a', 'Pdgfra', 'Pdgfrb', 'Pdk1', 'Pdk2', 
          'Pink1', 'Pjvk', 'Pla2r1', 'Plekha1', 'Pml', 'Pnpla8', 'Pnpt1', 'Ppargc1a', 'Ppargc1b', 'Ppia', 'Ppif', 'Ppp1r15a', 'Ppp1r15b', 'Prdx1', 'Prdx2', 'Prdx3', 'Prdx5', 'Prkaa1', 'Prkaa2', 
          'Prkcd', 'Prkch', 'Prkd1', 'Prkn', 'Prkra', 'Prr5l', 'Psap', 'Ptprk', 'Pycr1', 'Pycr2', 'Pyroxd1', 'Rack1', 'Rad52', 'Rbm11', 'Rbm4', 'Rela', 'Rest', 'Rhob', 'Ripk1', 'Ripk3', 'Rnf112', 
          'Rnf146', 'Rnf7', 'Romo1', 'Rps3', 'Rwdd1', 'Scly', 'Selenon', 'Selenos', 'Sesn2', 'Setx', 'Sfpq', 'Sigmar1', 'Sin3a', 'Sirpa', 'Sirt1', 'Sirt2', 'Slc11a2', 'Slc1a1', 'Slc25a24', 'Slc35a4', 
          'Slc4a11', 'Slc7a11', 'Slc8a1', 'Smpd3', 'Snca', 'Sod1', 'Sod2', 'Sod3', 'Sphk1', 'Src', 'Srxn1', 'Stat6', 'Stk24', 'Stk25', 'Stk26', 'Stx2', 'Stx4a', 'Tbc1d24', 'Tlr4', 'Tlr6', 'Tmed2', 
          'Tmem161a', 'Tnf', 'Tnfaip3', 'Trap1', 'Trem2', 'Trp53', 'Trp53inp1', 'Trpc6', 'Trpm2', 'Tsc1', 'Txn1', 'Txndc2', 'Uaca', 'Ubqln1', 'Ucp1', 'Vkorc1l1', 'Vnn1', 'Vrk2', 'Wnt1', 'Wnt16', 
          'Xbp1', 'Zc3h12a', 'Zfp277', 'Zfp580', 'Zfp622']  # GO:0043555 + GO:1900407


stress_filtered = ['Abl1', 'Akt1', 'Aldh2', 'Anxa1', 'App', 'Aqp1', 'Arl6ip5', 'Arntl', 'Atf4', 'Atp2a2', 'Axl', 'Bag5', 'Bcl2', 'Bmp4', 'Bnip3', 'Cfl1', 'Ctnnb1', 'Ddr2', 'Ddx3x', 'Ednra', 'Fbln5', 
                   'Fos', 'Foxo3', 'Foxp1', 'Fyn', 'Fzd1', 'Gpx1', 'Gsr', 'Hif1a', 'Hmox1', 'Hspb1', 'Hsph1', 'Jak2', 'Jun', 'Kdm6b', 'Lcn2', 'Mapk3', 'Mapkap1', 'Mcl1', 'Mgst1', 'Mmp2', 'Mmp3', 
                   'Nfe2l1', 'Nfe2l2', 'Nono', 'Nr4a2', 'P4hb', 'Park7', 'Pdcd10', 'Pdgfra', 'Pdgfrb', 'Ppia', 'Ppp1r15a', 'Prdx1', 'Prdx2', 'Prdx5', 'Psap', 'Rack1', 'Rela', 'Rhob', 'Romo1', 'Rwdd1', 
                   'Selenos', 'Sfpq', 'Sirpa', 'Smpd3', 'Sod1', 'Sod2', 'Srxn1', 'Stx4a', 'Tmed2', 'Tnfaip3', 'Trp53', 'Txn1', 'Uaca', 'Xbp1', 'Zfp622', ]

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=complement_genes_filtered, dendrogram=False, swap_axes=True, figsize=(10, 4), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=cytokine_interaction_genes_filtered, dendrogram=False, swap_axes=True, figsize=(10, 13), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=cytokine_release, dendrogram=False, swap_axes=True, figsize=(10, 1), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=ECM_assembly_disassembly_filtered, dendrogram=False, swap_axes=True, figsize=(10, 6), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=hypoxia_filtered, dendrogram=False, swap_axes=True, figsize=(10, 8), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=OXPHOS_filtered, dendrogram=False, swap_axes=True, figsize=(10, 10), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=glycolysis_filtered, dendrogram=False, swap_axes=True, figsize=(10, 5), vmax=0.85, categories_order=list(dict_fbs.keys()))

In [ ]:
sc.pl.matrixplot(adata_fb, groupby='cell_type', var_names=stress_filtered, dendrogram=False, swap_axes=True, figsize=(10, 11), vmax=0.85, categories_order=list(dict_fbs.keys()))

## Plot showing upregulation of glycolysis on Hilpda population

In [ ]:
sc.tl.score_genes(adata_fb, gene_list=glycolysis, ctrl_size=50, gene_pool=None, n_bins=25, score_name='score_glycolysis', random_state=0, copy=False, use_raw=None)

In [ ]:
sc.pl.umap(adata_fb, color=['cell_type', 'score_glycolysis'], ncols=2, frameon=False, cmap=magma)

Apply to the rest of paths

In [ ]:
for name, path in zip(['Complement', 'Cytokine', 'ECM', 'Hypoxia', 'OXPHOS', 'Glycolysis', 'Stress'], 
                      [complement_genes, cytokine_interaction_genes, ECM_assembly_disassembly, hypoxia, OXPHOS, glycolysis, stress]):
    
    sc.tl.score_genes(adata_fb, gene_list=path, ctrl_size=50, gene_pool=None, n_bins=25, score_name=f'score_{name}', random_state=0, copy=False, use_raw=None)

    
sc.pl.umap(adata_fb, color=['cell_type'] + [f'score_{name}' for name in ['Complement', 'Cytokine', 'ECM', 'Hypoxia', 'OXPHOS', 'Glycolysis', 'Stress']], ncols=3, frameon=False, cmap=magma)

In [ ]:
for name in ['Complement', 'Cytokine', 'ECM', 'Hypoxia', 'OXPHOS', 'Glycolysis', 'Stress']:
        sc.pl.violin(adata_fb, keys=f'score_{name}', groupby='cell_type', rotation=90)

## UMAP highlighting the expression of specific markers

In [ ]:
sc.pl.umap(adata_fb, color=['batch',  'Pdpn', 'C3', 'cell_type', 'C4b', 'Csf1'], ncols=3, frameon=False, cmap=magma, show=False)
plt.tight_layout()

In [ ]:
sc.pl.umap(adata_fb[adata_fb.obs['batch'].isin(['WT1', 'WT2'])], color=['batch', 'Rab37', 'Mmp9'], ncols=3, frameon=False, cmap=magma, show=False)
sc.pl.umap(adata_fb[adata_fb.obs['batch'].isin(['KO1', 'KO2'])], color=['batch', 'Rab37', 'Mmp9'], ncols=3, frameon=False, cmap=magma, show=False)